In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from magpylib.source.magnet import Box,Cylinder
from magpylib import Collection, displaySystem
from matplotlib.patches import Rectangle
import magpylib as mp

In [ ]:
VMAX = 2.4

# sensitivity of each sensor type
mVG = [
    5,      # A1324
    3.125,  # A1325
    2.5     # A1326
]

# ratio of G or mT at 35deg
RATIO_35DEG = np.sin(np.deg2rad(35))
G_mT = 0.1 # Gauss per mT

# mT required to hit VMAX at 35deg
mT_MAX = [(VMAX / (s / 1000) / RATIO_35DEG * G_mT)  for s in mVG]

# magnets
M_DIA = 6
M_HEIGHT = 3
M_SEP = 7 + M_HEIGHT
M_mT = 550

MAG_R1 = Rectangle((-M_DIA/2, -M_HEIGHT/2 - M_SEP/2), M_DIA, M_HEIGHT, fc='lightgray')
MAG_R2 = Rectangle((-M_DIA/2, -M_HEIGHT/2 + M_SEP/2), M_DIA, M_HEIGHT, fc='lightgray')



In [ ]:
# create magnets
s1 = Cylinder(mag=(0,0,M_mT), dim=(M_DIA, M_HEIGHT), pos=(0, 0, -M_SEP/2))
s2 = Cylinder(mag=(0,0,M_mT), dim=(M_DIA, M_HEIGHT), pos=(0, 0, M_SEP/2))

c = Collection(s1,s2)

In [ ]:
# calculate B-field on a grid
xs = np.linspace(-10,10,40)
zs = np.linspace(-10,10,40)

POS = np.array([(x,0,z) for z in zs for x in xs])
Bs = c.getB(POS).reshape(40,40,3)

ZPOS = np.array([(0,0,z) for z in zs])
Bz = c.getB(ZPOS)

In [ ]:
# create figure
fig = plt.figure(figsize=(16,16))
ax1 = fig.add_subplot(221, projection='3d')  # 3D-axis
ax2 = fig.add_subplot(222)                   # 2D-axis
ax3 = fig.add_subplot(223)
ax4 = fig.add_subplot(224)

# display system geometry on ax1
displaySystem(c, subplotAx=ax1, suppress=True)

# display field in xz-plane using matplotlib
X,Z = np.meshgrid(xs,zs)
U,V = Bs[:,:,0], Bs[:,:,2]
ax2.streamplot(X, Z, U, V)

# display contours for mT_MAX
ax3.contour(X, Z, np.abs(V), mT_MAX, colors=['red', 'green', 'blue'])
ax3.add_patch(MAG_R1)
ax3.add_patch(MAG_R2)

# show field strength along the Z axis
ax4.plot(zs, Bz[:,2])
ax4.plot(zs, np.full(len(Bz), mT_MAX[0]), color='red', label='A1324')
ax4.plot(zs, np.full(len(Bz), mT_MAX[1]), color='green', label='A1325')
ax4.plot(zs, np.full(len(Bz), mT_MAX[2]), color='blue', label='A1326')
ax4.grid(True)

plt.show()